In [ ]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [ ]:
# HuggingFace transformers 설치
!pip install transformers

     |████████████████████████████████| 1.5MB 17.3MB/s 
     |████████████████████████████████| 2.9MB 50.3MB/s 
     |████████████████████████████████| 890kB 49.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=1ec8bc062235fb0b5476604707e63f4e6458534e0c1e0cd73d43b7220157dd8c
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import pandas as pd
import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, ElectraForSequenceClassification, AdamW
from tqdm.notebook import tqdm

In [ ]:
# GPU 사용
device = torch.device("cuda") if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
class NewsDataset(Dataset):
  
  def __init__(self, data):
    self.dataset = data
    self.tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-small-v2-discriminator")

    print(self.dataset.describe())
  
  def __len__(self):
    return len(self.dataset)
  
  def __getitem__(self, idx):
    row = self.dataset.iloc[idx, 1:3].values
    text = row[0]
    y = row[1]

    inputs = self.tokenizer(
        text, 
        return_tensors='pt',
        truncation=True,
        max_length=256,
        pad_to_max_length=True,
        add_special_tokens=True
        )
    
    input_ids = inputs['input_ids'][0]
    attention_mask = inputs['attention_mask'][0]

    return input_ids, attention_mask, y

In [ ]:
df = pd.read_csv("news_data/news_train.csv")
dtls = df[['n_id', 'content', 'info']]

dtls

,n_id,content,info
0,NEWS02580,[이데일리 MARKETPOINT]15:32 현재 코스닥 기관 678억 순매도,0
1,NEWS02580,"""실적기반"" 저가에 매집해야 할 8월 급등유망주 TOP 5 전격공개",1
2,NEWS02580,"하이스탁론, 선취수수료 없는 월 0.4% 최저금리 상품 출시",1
3,NEWS02580,종합 경제정보 미디어 이데일리 - 무단전재 & 재배포 금지,0
4,NEWS09727,전국적인 소비 붐 조성에 기여할 예정,0
...,...,...,...
118740,NEWS01333,미 FDA 임상3상 허가 임박. 묻고 따블로 갈 바이오 황제주.,1
118741,NEWS01333,똑똑해진 소비자..한국도 이젠 소형차 시대,1
118742,NEWS01333,똑똑해진 소비자..한국도 이젠 소형차 시대,1
118743,NEWS01333,2020년 한국 TV 2대중 1대 인터넷 연결된다,1


In [ ]:
# train, validation, test set을 나눠주세요
from sklearn.model_selection import train_test_split

dataset_train, dataset_eval = train_test_split(dtls, test_size=0.1, random_state=123)

In [ ]:
train_dataset = NewsDataset(dataset_train)
val_dataset = NewsDataset(dataset_eval)


                info
count  106870.000000
mean        0.395293
std         0.488916
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max         1.000000
               info
count  11875.000000
mean       0.394695
std        0.488806
min        0.000000
25%        0.000000
50%        0.000000
75%        1.000000
max        1.000000


In [ ]:
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-small-v2-discriminator").to(device)

# 한번 실행해보기
# text, attention_mask, y = train_dataset[0]
# model(text.unsqueeze(0).to(device), attention_mask=attention_mask.unsqueeze(0).to(device))

Some weights of the model checkpoint at monologg/koelectra-small-v2-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-small-v2-discriminator and are newly initialized

In [ ]:
model.load_state_dict(torch.load("model.pt"))

FileNotFoundError: ignored

## train

In [ ]:
epochs = 10
batch_size = 128
optimizer = AdamW(model.parameters(), lr=1e-5)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=True)

In [ ]:
import copy

losses = []
accuracies = []

best_model_wts = copy.deepcopy(model.state_dict())
best_acc = 0.0

for i in range(epochs):
  total_loss = 0.0
  correct = 0
  total = 0
  batches = 0

  model.train()

  for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_loader):
    optimizer.zero_grad()
    y_batch = y_batch.to(device)
    y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
    loss = F.cross_entropy(y_pred, y_batch)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

    _, predicted = torch.max(y_pred, 1)
    correct += (predicted == y_batch).sum()
    total += len(y_batch)

    batches += 1
    if batches % 100 == 0:
      print("Batch Loss:", total_loss, "Accuracy:", correct.float() / total)
  
  model.eval()

  test_correct = 0
  test_total = 0

  for input_ids_batch, attention_masks_batch, y_batch in tqdm(val_loader):
    y_batch = y_batch.to(device)
    y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
    _, predicted = torch.max(y_pred, 1)
    test_correct += (predicted == y_batch).sum()
    test_total += len(y_batch)

  epoch_acc = test_correct.float() / test_total
  print("{} epoch Accuracy:".format(i), epoch_acc)
  
  if epoch_acc > best_acc:
    best_acc = epoch_acc
    best_model_wts = copy.deepcopy(model.state_dict())
  
  losses.append(total_loss)
  accuracies.append(correct.float() / total)
  print("Train Loss:", total_loss, "Accuracy:", correct.float() / total)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2179: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Batch Loss: 4.509746863506734 Accuracy: tensor(0.9866, device='cuda:0')
Batch Loss: 8.800289980135858 Accuracy: tensor(0.9870, device='cuda:0')
Batch Loss: 12.763301709666848 Accuracy: tensor(0.9873, device='cuda:0')
Batch Loss: 16.877170662395656 Accuracy: tensor(0.9871, device='cuda:0')
Batch Loss: 20.3419360043481 Accuracy: tensor(0.9875, device='cuda:0')
Batch Loss: 24.06004179175943 Accuracy: tensor(0.9876, device='cuda:0')
Batch Loss: 27.48906474886462 Accuracy: tensor(0.9877, device='cuda:0')
Batch Loss: 30.942323415074497 Accuracy: tensor(0.9879, device='cuda:0')




Accuracy: tensor(0.9900, device='cuda:0')
Train Loss: 32.15882581891492 Accuracy: tensor(0.9879, device='cuda:0')


Batch Loss: 3.110497873276472 Accuracy: tensor(0.9887, device='cuda:0')
Batch Loss: 5.896525257267058 Accuracy: tensor(0.9892, device='cuda:0')
Batch Loss: 8.79721223958768 Accuracy: tensor(0.9894, device='cuda:0')
Batch Loss: 11.810111686354503 Accuracy: tensor(0.9896, device='cuda:0')
Batch Loss: 14.392265670467168 Accuracy: tensor(0.9900, device='cuda:0')
Batch Loss: 17.245181140024215 Accuracy: tensor(0.9900, device='cuda:0')
Batch Loss: 19.8930777751375 Accuracy: tensor(0.9901, device='cuda:0')
Batch Loss: 22.51872006850317 Accuracy: tensor(0.9902, device='cuda:0')




Accuracy: tensor(0.9908, device='cuda:0')
Train Loss: 23.45533738238737 Accuracy: tensor(0.9902, device='cuda:0')


...


Accuracy: tensor(0.9944, device='cuda:0')
Train Loss: 8.110115511226468 Accuracy: tensor(0.9969, device='cuda:0')


Batch Loss: 0.642634229559917 Accuracy: tensor(0.9978, device='cuda:0')
Batch Loss: 1.458188770513516 Accuracy: tensor(0.9973, device='cuda:0')
Batch Loss: 2.245181583071826 Accuracy: tensor(0.9974, device='cuda:0')
Batch Loss: 3.097519103641389 Accuracy: tensor(0.9975, device='cuda:0')
Batch Loss: 3.873120340620517 Accuracy: tensor(0.9975, device='cuda:0')
Batch Loss: 4.779446391519741 Accuracy: tensor(0.9975, device='cuda:0')
Batch Loss: 5.740266709544812 Accuracy: tensor(0.9974, device='cuda:0')
Batch Loss: 6.3389885163196595 Accuracy: tensor(0.9974, device='cuda:0')




Accuracy: tensor(0.9947, device='cuda:0')
Train Loss: 6.697079789402778 Accuracy: tensor(0.9974, device='cuda:0')


Batch Loss: 0.7816549769195262 Accuracy: tensor(0.9972, device='cuda:0')
Batch Loss: 1.6660094233520795 Accuracy: tensor(0.9971, device='cuda:0')
Batch Loss: 2.3854760465183062 Accuracy: tensor(0.9973, device='cuda:0')
Batch Loss: 3.128119043118204 Accuracy: tensor(0.9973, device='cuda:0')
Batch Loss: 4.0214547993819 Accuracy: tensor(0.9974, device='cuda:0')
Batch Loss: 4.723672985302983 Accuracy: tensor(0.9974, device='cuda:0')
Batch Loss: 5.3848266212735325 Accuracy: tensor(0.9974, device='cuda:0')
Batch Loss: 6.0527838338311994 Accuracy: tensor(0.9975, device='cuda:0')




Accuracy: tensor(0.9944, device='cuda:0')
Train Loss: 6.27601778593089 Accuracy: tensor(0.9975, device='cuda:0')


In [ ]:
losses, accuracies

([32.15882581891492,
  23.45533738238737,
  18.620169282192364,
  15.693005793727934,
  12.943048821412958,
  11.15986925989273,
  9.38576854681014,
  8.110115511226468,
  6.697079789402778,
  6.27601778593089],
 [tensor(0.9879, device='cuda:0'),
  tensor(0.9902, device='cuda:0'),
  tensor(0.9923, device='cuda:0'),
  tensor(0.9937, device='cuda:0'),
  tensor(0.9949, device='cuda:0'),
  tensor(0.9957, device='cuda:0'),
  tensor(0.9963, device='cuda:0'),
  tensor(0.9969, device='cuda:0'),
  tensor(0.9974, device='cuda:0'),
  tensor(0.9975, device='cuda:0')])

## eval

In [ ]:
# 모델 저장하기
torch.save(model.state_dict(), "news_result/koelectra_e3_b128.pt")

## test

In [ ]:
df = pd.read_csv("news_data/news_test.csv")
test_dtls = df[['id', 'content', 'ord']]

test_dtls

,id,content,ord
0,NEWS00237_1,마이크로 LED TV 장비 양산 돌입- 전방업체 투자 확대로 본업도 호조연일 '신고가',1
1,NEWS00237_2,[이데일리 김대웅 기자] 반도체 장비 업체 코세스(089890)의 기술력이 마이크로...,2
2,NEWS00237_3,최근 대형 업체들과 거래를 맺고 관련 장비들의 양산에 돌입하면서 주가도 연일 신고가...,3
3,NEWS00237_4,TV를 필두로 올해부터 마이크로 LED의 시대가 본격적으로 개화할 것으로 예상되면서...,4
4,NEWS00237_5,"코세스는 반도체 장비를 제조, 판매하는 업체로 지난 2006년 11월 코스닥 시장에...",5
...,...,...,...
142560,NEWS09482_72,주식시장의 역사를 다시 쓸 역대급 종목. 목표 1700% 이상.,72
142561,NEWS09482_73,똑똑해진 소비자..한국도 이젠 소형차 시대,73
142562,NEWS09482_74,똑똑해진 소비자..한국도 이젠 소형차 시대,74
142563,NEWS09482_75,2020년 한국 TV 2대중 1대 인터넷 연결된다,75


In [ ]:
test_dataset = NewsDataset(test_dtls)

test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True)

                 ord
count  142565.000000
mean       32.524196
std        38.343797
min         1.000000
25%         8.000000
50%        17.000000
75%        45.000000
max       348.000000


In [ ]:
model.eval()

pred = []

for input_ids_batch, attention_masks_batch, y_batch in tqdm(test_loader):
  y_batch = y_batch.to(device)
  y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
  _, predicted = torch.max(y_pred, 1)
  pred += list(predicted.cpu().clone().numpy())
  test_correct += (predicted == y_batch).sum()
  test_total += len(y_batch)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2179: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
result = pd.concat([test_dtls['id'], pd.DataFrame(pred)], ignore_index=True, axis=1)
result = result.rename(columns={0:'id',  1:'info'})
result

,0,1
0,NEWS00237_1,0
1,NEWS00237_2,0
2,NEWS00237_3,1
3,NEWS00237_4,1
4,NEWS00237_5,0
...,...,...
142560,NEWS09482_72,0
142561,NEWS09482_73,0
142562,NEWS09482_74,1
142563,NEWS09482_75,1


In [ ]:
result.to_csv('news_result/M_koelectra_e10_b128.csv', index=False)